In [ ]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import numpy as np
import re

def preprocess(filepath):

  preprocessed = []

  with open(filepath, 'r') as file:
    text = ' '.join(file.readlines())
    text = re.split(r'\s?#?\d+.+.\n\s+', text)
    text = [line.split('\n') for line in text]

    for sent in text:
      new_sent = []
      for line in sent:
        line = re.sub('<TAB>', ',', line)
        line = line.split(' , ')

        if line != [''] and line != [' ']:
          new_sent.append(line)

      preprocessed.append(new_sent)

    #TODO peut-etre supprimer '(speculative)' flag car il est pas toujours respecté

  return preprocessed

def pad(annotations1, annotations2):

    #j'aurais pu faure una matrice de zero et la remplir mais j'ai eu l'idée trop tard

    padded1 = []
    padded2 = []

    for s1, s2 in zip(annotations1, annotations2):

        max_len = max(len(s1), len(s2))
        for i in range (max_len):
            sent1=[]
            sent2=[]
            if i>=len(s1):
                sent1=["" for _ in range (len(s2[i]))]
            else:
                sent1=s1[i]

            if i>=len(s2):
                sent2=["" for _ in range (len(s1[i]))]
            else:
                sent2=s2[i]

            if i < len(s1) and  i < len(s2) and len(sent1) < len(sent2):
                while len(sent1) < len(sent2):
                    sent1.append('')

            if i < len(s1) and  i < len(s2) and len(sent2) < len(sent1):
                while len(sent2) < len(sent1):
                    sent2.append('')

            padded1.append(sent1)
            padded2.append(sent2)



    return padded1, padded2

In [ ]:
ann1=[[], [['He / (Chomsky)', 'created', 'the universal grammar theory'], [' He / (Chomsky)', 'created', 'the generative grammar theory'], [' He / (Chomsky)', 'created', 'the Chomsky hierarchy'], [' He / (Chomsky)', 'created', 'the the minimalist program'], [' He / (Chomsky)', 'created', 'the universal grammar theory', 'the generative grammar'], [' theory', 'the Chomsky hierarchy and', 'the minimalist program']], [['was particularly critical of the work of B. F. Skinner .'], [' Chomsky', 'played a pivotal role in', 'the decline of', 'linguistic behaviorism'], [' Chomsky', 'was particularly critical of', 'the work of', 'B. F. Skinner .'], [' Chomsky', 'criticised', 'the work of', 'B. F. Skinner .']], [['act of American imperialism', 'in']], [['anti-war essay " The Responsibility of Intellectuals " .'], [' [Chomsky]', 'was', 'an opponent of', 'U.S. involvement in', 'the Vietnam War'], [" _comment: est-ce qu'on garde l'adjectif dans les cas pareils? Genre outspoken_"], [' He / (Chomsky)', 'saw', 'the Vietnam War', 'as an act of', 'American imperialism'], [' Chomsky', 'rose to national attention for', 'his / (Chomsky\'s) anti-war essay " The Responsibility of Intellectuals "', 'in']], [['[Chomsky]', '[wrote]', '" The Responsibility of Intellectuals "']], [["activism and placed on President Richard Nixon 's list of political opponents ."], [' He / (Chomsky)', 'was arrested for', "his / (Chomsky's) activism", 'multiple times'], [' He / (Chomsky)', 'was placed on', "President Richard Nixon 's list of", 'political opponents'], [' [Richard Nixon]', '[had] political opponents'], [' [Richard Nixon]', '[was a] president']], [['involved in the linguistics wars .'], [' He / (Chomsky)', 'became involved in', 'the linguistics wars'], [' He / (Chomsky)', '[has been] expanding', "his / (Chomsky's) work in", 'linguistics', 'over subsequent decades / (after']], [], [['model of media criticism in Manufacturing Consent', 'and worked to expose the'], [' Indonesian occupation of East Timor .'], [' Chomsky', 'collaborated with', 'Edward S. Herman'], [' Chomsky', 'articulated', 'the propaganda model of', 'media criticism', 'in Manufacturing Consent'], [" _comment: ajouter 'later' c'est une idée à la con c ça?_"], [' Chomsky', 'worked to expose', 'the Indonesian occupation of', 'East Timor']], [['denial', 'generated significant controversy in the Faurisson affair of the']], [["His / (Chomsky's) defense", 'of unconditional freedom of', 'speech', 'generated significant controversy in', 'the Faurisson affair', 'of the']], [['[Chomsky]', '[defended]', 'freedom of speech'], [" His / (Chomsky's) defense", 'included', 'Holocaust denial '], [' _comment: peut-être ça a pas de sens les gars je suis défonsé je sais meme pas écrire défonsé_']], [["Chomsky 's commentary on", 'the Cambodian genocide', 'generated controversy'], [' [Chomsky]', '[commented] on', 'the Cambodian genocide']], [['activism', 'including opposing the']], [['movement .'], [' [Chomsky]', '[retired] from', 'teaching', 'at MIT'], [' He / (Chomsky)', 'has continued', "his / (Chomsky's) vocal political activism"], [' He / (Chomsky)', '[opposed] the invasion <TAB >of Iraq', 'in']], [['He / (Chomsky)', '[supported]', 'the Occupy movement']], [['Chomsky', 'began teaching at', 'the University of Arizona', 'in']], []]
ann2=[[], [['He /(Avram Noam Chomsky)', 'created  or co-created', 'the universal grammar theory'], [' He /(Avram Noam Chomsky)', 'created  or co-created', 'the generative grammar theory'], [' He /(Avram Noam Chomsky)', 'created  or co-created', 'the Chomsky hierarchy '], [' He /(Avram Noam Chomsky)', 'created  or co-created', 'the minimalist program']], [['Chomsky/ (Avram Noam Chomsky)', 'played a pivotal role', 'in the decline of linguistic behaviorism '], [' Chomsky/ (Avram Noam Chomsky)', 'was crticial of', 'the work of B. F. Skinner /(Burrhus Frederic Skinner)']], [['[Avram Noam Chomsky]', '[was] an opponent of ', 'U.S. involvement in the Vietnam war'], [' [Avram Noam Chomsky]', '[was] saw', 'U.S. involvement in the Vietnam war', 'as an act of American imperialism'], [' Chomsky/ (Avram Noam Chomsky)', '[wrote]', '"The Responsibility of Intellectuals" '], [' Chomsky/ (Avram Noam Chomsky)', '[wrote]', '"The Responsibility of Intellectuals"', 'in']], [['Chomsky/ (Avram Noam Chomsky)', 'rose to national attention', 'for his anti-war essay "The Responsibility of Intellectuals" '], [' "The Responsibility of Intellectuals"', '[is]', '[an] anti-war essay']], [['[Avram Noam Chomsky]', '[was] associated with', 'the New Left'], [' He/ (Avram Noam Chomsky)', 'was arrested '], [' He/ (Avram Noam Chomsky)', 'was arrested', 'multiple times'], [' He/ (Avram Noam Chomsky)', 'was arrested', 'multiple times', 'for his activism '], [' He/ (Avram Noam Chomsky)', 'was placed', "on President Richard Nixon 's list of political opponents"], [' Richard Nixon', '[was]', 'president'], [' Richard Nixon', '[had]', 'political opponents']], [['[Avram Noam Chomsky]', '[was] involved in', 'the linguistic wars'], [' [Avram Noam Chomsky]', '[was] involved in', 'the linguistic wars', 'while expanding his work in linguistics'], [' He/ (Avram Noam Chomsky)', '[expanded] his work', 'in linguistics over [several] decades']], [['Chomsky/ (Avram Noam Chomsky)', '[collaborated] with', 'Edward S. Herman'], [' Chomsky/ (Avram Noam Chomsky) [and] Edward S. Herman', 'articulated', 'the propaganda model of media criticism '], [' Chomsky/ (Avram Noam Chomsky) [and] Edward S. Herman', 'articulated', 'the propaganda model of media criticism', 'in Manufacturing Consent'], [' Chomsky/ (Avram Noam Chomsky) [and] Edward S. Herman', 'worked to expose', 'the Indonesian occupation of East Timor'], [' Indonesia', '[occupied]', 'East Timor']], [['[Avram Noam Chomsky]', '[defended]', 'unconditioanl freedom of speech'], [' unconditioanl freedom of speech', '[includes]', '[the] Holocaust denial'], [" His/(Avram Noam Chomsky's) defense of unconditional freedom of speech", 'generated', 'controversy '], [" His/(Avram Noam Chomsky's) defense of unconditional freedom of speech", 'generated', 'controversy', 'in the Faurisson affair of the']], [], [["Chomsky's/ (Avram Noam Chomsky's) commentary on the Cambodian genocide", 'generated controversy']], [['he/ (Avram Noam Chomsky)', '[retired] from', 'active teaching at MIT'], [' he/ (Avram Noam Chomsky)', '[practiced]', 'vocal political activism '], [' he/ (Avram Noam Chomsky)', 'continued', 'his vocal political activism', 'after  retiring from active teaching at MIT'], [" his/(Avram Noam Chomsky's) activism", '[included]', 'opposing the']], [["his/(Avram Noam Chomsky's) activism", '[supported]', 'the Occupy movement']], [['Chomsky/ (Avram Noam Chomsky)', '[teached]', 'at the University of Arizona'], [' Chomsky/ (Avram Noam Chomsky)', 'began teaching', 'at the University of Arizona in 2017']]]


In [ ]:
from itertools import chain


x, y = pad(ann1, ann2)

print(x)
print(y)

[['He / (Chomsky)', 'created', 'the universal grammar theory'], [' He / (Chomsky)', 'created', 'the generative grammar theory'], [' He / (Chomsky)', 'created', 'the Chomsky hierarchy'], [' He / (Chomsky)', 'created', 'the the minimalist program'], [' He / (Chomsky)', 'created', 'the universal grammar theory', 'the generative grammar'], [' theory', 'the Chomsky hierarchy and', 'the minimalist program'], ['was particularly critical of the work of B. F. Skinner .', '', ''], [' Chomsky', 'played a pivotal role in', 'the decline of', 'linguistic behaviorism'], [' Chomsky', 'was particularly critical of', 'the work of', 'B. F. Skinner .'], [' Chomsky', 'criticised', 'the work of', 'B. F. Skinner .'], ['act of American imperialism', 'in', ''], ['', '', '', ''], ['', '', ''], ['', '', '', ''], ['anti-war essay " The Responsibility of Intellectuals " .', '', ''], [' [Chomsky]', 'was', 'an opponent of', 'U.S. involvement in', 'the Vietnam War'], [" _comment: est-ce qu'on garde l'adjectif dans le

In [ ]:
# calculer sur toute l'annotation
x = list(chain.from_iterable(x))
y = list(chain.from_iterable(y))

print(x)
print(y)
print("{:.2f}".format(sklearn.metrics.cohen_kappa_score(x, y)))

['He / (Chomsky)', 'created', 'the universal grammar theory', ' He / (Chomsky)', 'created', 'the generative grammar theory', ' He / (Chomsky)', 'created', 'the Chomsky hierarchy', ' He / (Chomsky)', 'created', 'the the minimalist program', ' He / (Chomsky)', 'created', 'the universal grammar theory', 'the generative grammar', ' theory', 'the Chomsky hierarchy and', 'the minimalist program', 'was particularly critical of the work of B. F. Skinner .', '', '', ' Chomsky', 'played a pivotal role in', 'the decline of', 'linguistic behaviorism', ' Chomsky', 'was particularly critical of', 'the work of', 'B. F. Skinner .', ' Chomsky', 'criticised', 'the work of', 'B. F. Skinner .', 'act of American imperialism', 'in', '', '', '', '', '', '', '', '', '', '', '', '', 'anti-war essay " The Responsibility of Intellectuals " .', '', '', ' [Chomsky]', 'was', 'an opponent of', 'U.S. involvement in', 'the Vietnam War', " _comment: est-ce qu'on garde l'adjectif dans les cas pareils? Genre outspoken_",

In [ ]:
# calculer sentence par sentence et trouver la moyenne
import sklearn.metrics

x, y = pad(ann1, ann2)
scores = []

for sent1, sent2 in zip(x,y):
  score = sklearn.metrics.cohen_kappa_score(sent1, sent2)
  scores.append(score)

  # decommenter pour voir les phrases et les scores precises
  # print(sent1, sent2)
  # print("{:.2f}".format(score))

print('average kappa score: ', "{:.3f}".format(sum(scores)/len(scores)))

average kappa score:  0.009


In [ ]:
# ici in pourra calculer le score entre l'annotation
from sklearn.metrics import f1_score

# f1 = f1_score(y_true= ##, y_pred=##)
print(f1)

In [ ]:
# ce truc est horrible les amis il veut un format chelou
# il y a un exemple mais bof https://gist.github.com/cbuntain/9dd7e42d5d8ab34609162410e06f3270

# from nltk.metrics.agreement import AnnotationTask
# t = AnnotationTask(data=[x, y])

In [ ]:
!java -version

In [ ]:
!pip install -v stanford_openie


In [ ]:
from openie import StanfordOpenIE

# https://stanfordnlp.github.io/CoreNLP/openie.html#api
# Default value of openie.affinity_probability_cap was 1/3.
properties = {
    'openie.affinity_probability_cap': 1 / 3,
}

with StanfordOpenIE(properties=properties) as client:
    text = 'Barack Obama was born in Hawaii. Richard Manning wrote this sentence.'
    print('Text: %s.' % text)
    for triple in client.annotate(text):
        print('|-', triple)

    graph_image = 'graph.png'
    client.generate_graphviz_graph(text, graph_image)
    print('Graph generated: %s.' % graph_image)

    with open('corpus/pg6130.txt', encoding='utf8') as r:
        corpus = r.read().replace('\n', ' ').replace('\r', '')

    triples_corpus = client.annotate(corpus[0:5000])
    print('Corpus: %s [...].' % corpus[0:80])
    print('Found %s triples in the corpus.' % len(triples_corpus))
    for triple in triples_corpus[:3]:
        print('|-', triple)
    print('[...]')

In [ ]:
from nltk.metrics.distance import edit_distance
from nltk.metrics import agreement

def edit_distance_no_zero (sent_a, sent_b):
    dist = edit_distance(sent_a, sent_b)  / max(len(sent_a), len(sent_b)) + 0.0001
    return dist

scorer = agreement.AnnotationTask(data = [["Alina", 1, 'abcd'], ["Eleonore", 1, 'abc'], ["Alina", 1, 'abc'], ["Eleonore", 1, 'abcd']] ,distance=edit_distance_no_zero)

In [ ]:
scorer.kappa()

1.2501

In [ ]:
import sklearn

a = ["the Cambodian genocide" ]
b = ["at the University of Arizona in 2017" ]


dist = sklearn.metrics.cohen_kappa_score(a, b, weights='quadratic', sample_weight = [1-edit_distance_no_zero(tupa, tupb) for tupa, tupb in zip(a, b)])
print(dist)

print(x)
print(y)
print("{:.2f}".format(sklearn.metrics.cohen_kappa_score(x, y,  weights='linear', sample_weight = [1-edit_distance_no_zero(tupa, tupb) for tupa, tupb in zip(x, y)])))

0.0
['He / (Chomsky)', 'created', 'the universal grammar theory', ' He / (Chomsky)', 'created', 'the generative grammar theory', ' He / (Chomsky)', 'created', 'the Chomsky hierarchy', ' He / (Chomsky)', 'created', 'the the minimalist program', ' He / (Chomsky)', 'created', 'the universal grammar theory', 'the generative grammar', ' theory', 'the Chomsky hierarchy and', 'the minimalist program', 'was particularly critical of the work of B. F. Skinner .', '', '', ' Chomsky', 'played a pivotal role in', 'the decline of', 'linguistic behaviorism', ' Chomsky', 'was particularly critical of', 'the work of', 'B. F. Skinner .', ' Chomsky', 'criticised', 'the work of', 'B. F. Skinner .', 'act of American imperialism', 'in', '', '', '', '', '', '', '', '', '', '', '', '', 'anti-war essay " The Responsibility of Intellectuals " .', '', '', ' [Chomsky]', 'was', 'an opponent of', 'U.S. involvement in', 'the Vietnam War', " _comment: est-ce qu'on garde l'adjectif dans les cas pareils? Genre outspoke

In [ ]:
[1-edit_distance_no_zero(tupa, tupb) for tupa, tupb in zip(a, b)]

[0.9999, 0.9544454545454546, 0.9999]

In [ ]:
!pip install --upgrade pip setuptools
!pip install pyiextract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00
  Using cached setuptools-69.0.3-py3-none-any.whl (819 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.4/208.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.8/729.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !pip install urllib3 --upgrade
!pip install opennre

ERROR: Could not find a version that satisfies the requirement opennre (from versions: none)
ERROR: No matching distribution found for opennre


In [ ]:
import json
from pyiextract import pipeline

# from pyiextract.blinknormaliser import BlinkNormaliser
from pyiextract.coreferencenormaliser import CoreferenceNormaliser
from pyiextract.coreferencereducer import CoreferenceReducer
from pyiextract.englishnormaliser import EnglishNormaliser
from pyiextract.llmextractor import LLMExtractor
from pyiextract.nerreducer import NERReducer
from pyiextract.oiextractor import OIExtractor
from pyiextract.pipeline import Pipeline
from pyiextract.subjectivityreducer import SubjectivityReducer
from pyiextract.svoextractor import SVOExtractor

pipeline = Fullpipeline.fullpipeline([EnglishNormaliser(), CoreferenceNormaliser()], [SVOExtractor(), OIExtractor()],[CoreferenceReducer(), NERReducer(), SubjectivityReducer()])
doc = pipeline.extract("Avram Noam Chomsky ( born December 7 , 1928 ) is an American professor and public intellectual known for his work in linguistics , political activism , and social criticism .")
print(json.dumps([str(x) for x in doc.triples()]))

Output()

ConfigValidationError: 

Config validation error

tagger -> label_smoothing   extra fields not permitted

{'nlp': <spacy.lang.en.English object at 0x7a7f55e57850>, 'name': 'tagger', 'label_smoothing': 0.0, 'model': {'@architectures': 'spacy.Tagger.v2', 'nO': None, 'normalize': False, 'tok2vec': {'@architectures': 'spacy.Tok2VecListener.v1', 'width': 96, 'upstream': 'tok2vec'}}, 'neg_prefix': '!', 'overwrite': False, 'scorer': {'@scorers': 'spacy.tagger_scorer.v1'}, '@factories': 'tagger'}

In [ ]:
!gdown 1QARlneZmqNsJhwTA3mFa9MGMh8CNm8QJ

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 151, in main
    filename = download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 203, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'


In [ ]:
import numpy

def get_matches(tuple1, tuple2):
    """
    returns nb of matches between tuple1 and tuple2 (token level)
    returns 0 if there is an argument without token match.
    """
    if len(tuple1) != len(tuple2):
        return 0
    total_matches = 0

    for arg_tuple1, arg_tuple2 in zip(tuple1, tuple2):
        arg_tuple1 = re.sub('\\.*\[.+\]', '', arg_tuple1)
        arg_tuple2 = re.sub('\\.*\[.+\]', '', arg_tuple2)

        arg_tuple1 = arg_tuple1.split(" ")
        arg_tuple2 = arg_tuple2.split(" ")
        matches = [value for value in arg_tuple1 if value in arg_tuple2]

        if len(matches) == 0:
            return 0

        total_matches += len(matches)

    return total_matches


def F1_scorer(annotations_pred, annotations_gold , weighted=False):
    """
    annotations_pred: list[list[list[str]]] all annotated sentences from annotator, each sentence is a list of list of strings (=a list of tuples)
    annotations_gold: list[list[list[str]]] all annotated sentences from gold corpus, each sentence is a list of list of strings (=a list of tuples)
    """

    precision_sys_numerator = 0
    precision_sys_denominator = 0
    recall_sys_numerator = 0
    recall_sys_denominator = 0

    #work on each sentence (list of tuples)
    for sentence_preds, sentence_golds in zip(annotations_pred, annotations_gold):

        precision_sentence_numerator = 0
        precision_sentence_denominator = 0
        recall_sentence_numerator = 0
        recall_sentence_denominator = 0

        while len(sentence_preds) > 0 and len(sentence_golds) > 0:

            #get for each predicted tuple its lenght
            pred_lengths = [len(" ".join(tup).split(" ")) for tup in sentence_preds]
            gold_lengths = [len(" ".join(tup).split(" ")) for tup in sentence_golds]
            f1_sentence = []

            #get for EACH predicted tuple its F1 score with EACH gold tuple:
            for index_pred, pred_tuple in enumerate(sentence_preds):
                f1_tuple = []
                for index_gold, gold_tuple in enumerate(sentence_golds):
                    ##  get number of matching tokens between all args of predicted tuple and eall args of all gold tuples
                    matches = get_matches(pred_tuple, gold_tuple)

                    ##  get recall and precision with each tuple
                    recall = matches / gold_lengths[index_gold]
                    precision = matches / pred_lengths[index_pred]

                    ## get F1 scores
                    if (precision+recall == 0):
                        f1_tuple.append(0)
                    else:
                        f1_tuple.append((2 * precision * recall) / (precision + recall))

                f1_sentence.append(f1_tuple)

            # and get argmax (delete predicted tuple and corresponding gold tuple)
            pred_index = numpy.argmax([max(tuples_f1) for tuples_f1 in f1_sentence])
            gold_index = numpy.argmax(f1_sentence[pred_index])

            precision_sentence_numerator += get_matches(sentence_preds[pred_index], sentence_golds[gold_index])
            recall_sentence_numerator += get_matches(sentence_preds[pred_index], sentence_golds[gold_index])
            precision_sentence_denominator += pred_lengths[pred_index]
            recall_sentence_denominator += gold_lengths[gold_index]


            sentence_preds.pop(pred_index)
            sentence_golds.pop(gold_index)

            pred_lengths.pop(pred_index)
            gold_lengths.pop(gold_index)

        #increase sys numerators and denominators
        precision_sys_numerator += precision_sentence_numerator
        precision_sys_denominator += precision_sentence_denominator
        recall_sys_numerator += recall_sentence_numerator
        recall_sys_denominator += recall_sentence_denominator

    precision = precision_sys_numerator / precision_sys_denominator
    recall = recall_sys_numerator / recall_sys_denominator

    #return overall F1
    return (2 * precision * recall ) / (precision + recall)


preds = [[["a a a","b b b", "c c c"], ["a a a", "b b b", "c d d"], ["b b b", "c c c", "g f r", "t t t"]]]
golds = [[["a a a", "a b a", "d g g"], ["a a a","b b b", "c c c"]]]

F1_scorer(preds, golds)

0.9444444444444444

In [ ]:


f1 = (2*0.5*0.5) / (0.5 + 0.5)
f2 = (2*0.4*0.3) / (0.4 + 0.3)

p = 0.4 + 0.5
r = 0.5 + 0.3

F = ((2*p*r) / (p+r) )/2
print("F:", F)
print("summed:", (f1+f2)/2)

F: 0.4235294117647059
summed: 0.42142857142857143


In [ ]:
import re
arg_tuple1 = "Chomsky / ( A A A) "
arg_tuple1 = re.sub('/.*\(.+\)', '', arg_tuple1)
print(arg_tuple1)

Chomsky 
